# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Apache Cassandra 

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class': 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In [8]:
# Query 1:  Give me the artist, song title and songs's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
# Description: Here the primary key has two fields: sessionid is the partitioning key and iteminsession is the clustering key \
# Partioning is done by iteminsession and within that partition, rows are ordered by iteminsession.
query1 = "CREATE TABLE IF NOT EXISTS artist_song_length_session"
query1 = query1 + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY (sessionid, iteminSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO artist_song_length_session (sessionid, iteminSession, artist, song, length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
query1 = "SELECT artist, song, length FROM artist_song_length_session WHERE sessionid = 338 AND iteminSession = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
# Query 2: Give me only the following: name of artist, song (sorted by iteminsesession) and user (first and last name)\
# for userid = 10, sessionid = 182
# Description: Here the primary key has two fields: userid and sessionid are both partitioning keys \
# Partitioning is done by userid and sessionid and within that partition, rows are sorted by iteminsession
query2 = "CREATE TABLE IF NOT EXISTS artist_song_user_session"
query2 = query2 + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, \
                    PRIMARY KEY ((userid, sessionid), iteminsession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO artist_song_user_session (userid, sessionid, iteminsession, artist, song, firstname, lastname)" 
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

In [12]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Description: Here the primary key has two fields: song is the partitioning key and userid is the clustering key \
# Partioning is done by song and within that partition, rows are ordered by userid.
query3 = "CREATE TABLE IF NOT EXISTS user_session"
query3 = query3 + "(song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"
try:
    session.execute(query3)
except Exception as e:
    print(e)
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query3 = "INSERT INTO user_session (song, userid, firstname, lastname)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query3, ( line[9], int(line[10]), line[1], line[4]))

In [13]:
query2 = "SELECT artist, song, firstname, lastname FROM artist_song_user_session WHERE userid = 10 AND sessionId = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
query3 = "SELECT firstname, lastname FROM user_session WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [15]:
query1 = "drop table artist_song_length_session"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "drop table artist_song_user_session"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "drop table user_session"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

In [16]:
session.shutdown()
cluster.shutdown()